In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
avs = np.array([0,1,2,3,5,10,20])
x = []
for av in avs:
    x.append(np.load("../data/GDR3/gaiadr3_%dext.npy" %(av)))
print(len(x[0]),x[0].dtype)

8102858 [('meh_ini', '<f8'), ('log_age', '<f8'), ('m_ini', '<f8'), ('m_act', '<f8'), ('log_lum', '<f8'), ('log_teff', '<f8'), ('log_grav', '<f8'), ('gaia_g', '<f8'), ('gaia_bpbr', '<f8'), ('gaia_bpft', '<f8'), ('gaia_rp', '<f8'), ('gaia_rvs', '<f8')]


In [3]:
for name in x[0].dtype.names:
    print(name,x[0][name],len(np.unique(x[0][name])))

meh_ini [-1.49213 -1.49213 -1.49213 ...  0.33631  0.33631  0.33631] 75
log_age [ 6.6      6.6      6.6     ... 10.12004 10.12004 10.12004] 177
m_ini [0.09       0.0956148  0.1        ... 5.80000019 5.99999952 6.19999933] 6276478
m_act [0.09  0.096 0.1   ... 0.94  0.961 0.962] 53226
log_lum [-1.531 -1.502 -1.481 ... -4.601 -4.603 -4.604] 10836
log_teff [3.5276 3.5305 3.5326 ... 3.6298 3.6321 3.6319] 27662
log_grav [3.986 3.995 4.001 ... 8.484 8.505 8.505] 12072
gaia_g [ 9.192  9.106  9.043 ... 16.513 16.507 16.51 ] 52080
gaia_bpbr [10.284 10.181 10.108 ... 17.284 17.27  17.273] 67368
gaia_bpft [10.319 10.217 10.143 ... 17.324 17.31  17.313] 67036
gaia_rp [ 8.163  8.085  8.027 ... 15.66  15.66  15.662] 49227
gaia_rvs [ 7.692  7.62   7.566 ... 15.318 15.323 15.325] 43937


In [4]:
def return_index_feh(feh):
        dfeh = 0.05
        offset = 1.5
        return(int((feh+offset)/dfeh))
min_value = min(x[0]['meh_ini'])
max_value = max(x[0]['meh_ini'])
min_index_feh = return_index_feh(min_value)
max_index_feh = return_index_feh(max_value)
stretch = max_index_feh - min_index_feh
print('teff values in parsec')
print('teff min max value: ', min_value, max_value )
print('teff min max index: ', min_index_feh, max_index_feh)
print('dimension cut into %d pieces' %(stretch))

def return_index_teff(teff):
        dteff = 0.01
        return(int(teff/dteff))
min_value = min(x[0]['log_teff'])
max_value = max(x[0]['log_teff'])
min_index_teff = return_index_teff(min_value)
max_index_teff = return_index_teff(max_value)
stretch = max_index_teff - min_index_teff
print('teff values in parsec')
print('teff min max value: ', min_value, max_value )
print('teff min max index: ', min_index_teff, max_index_teff)
print('dimension cut into %d pieces' %(stretch))

def return_index_lum(lum):
        dlum = 0.02
        offset = 5.
        return(int((lum+offset)/dlum))

min_value = min(x[0]['log_lum'])
max_value = max(x[0]['log_lum'])
min_index_lum = return_index_lum(min_value)
max_index_lum = return_index_lum(max_value)
stretch = max_index_lum - min_index_lum
print('lum values in parsec')
print('lum min max value: ', min_value, max_value )
print('lum min max index: ', min_index_lum, max_index_lum )
print('dimension cut into %d pieces' %(stretch))

def return_index(feh,teff,lum):
        """
        feh given in dex
        teff given in log teff
        lum given in log lum
        """
        index_feh = return_index_feh(feh)
        if index_feh < min_index_feh:
            index_feh = min_index_feh
        elif index_feh > max_index_feh:
            index_feh = max_index_feh

        index_teff = return_index_teff(teff)
        if index_teff < min_index_teff:
            index_teff = min_index_teff
        elif index_teff > max_index_teff:
            index_teff = max_index_teff
        index_teff *= 1000

        index_lum = return_index_lum(lum)
        if index_lum > max_index_lum:
            index_lum = max_index_lum
        elif index_lum < min_index_lum:
            index_lum = min_index_lum
        index_lum *= 1000 * 1000
        assert(index_feh >= 0)
        assert(index_teff >= 0)
        assert(index_lum >= 0)
        return (index_feh + index_teff + index_lum)

teff values in parsec
teff min max value:  -1.49213 0.33631
teff min max index:  0 36
dimension cut into 36 pieces
teff values in parsec
teff min max value:  1.666 5.6843
teff min max index:  166 568
dimension cut into 402 pieces
lum values in parsec
lum min max value:  -4.604 6.24
lum min max index:  19 562
dimension cut into 543 pieces


In [5]:
print('indexing isochrones')
indizes = np.zeros(shape=(len(x[0])),dtype = np.int32)
for i in range(len(x[0])):
    indizes[i] = return_index(x[0]["meh_ini"][i],x[0]["log_teff"][i],x[0]["log_lum"][i])

indexing isochrones


In [6]:
from astropy.io import fits
y = fits.getdata("../../../Galaxia_wrap-master/output/mock_cat_old_gdr3_isochrones/GDR2mock_20.7Gmag.fits")
print(len(y))
print('indexing galaxia')
indizes_g = np.zeros(len(y),dtype = np.int32)
for i in range(len(y)):
    indizes_g[i] = return_index(y['feh'][i],np.log10(y['teff'][i]),y['lum'][i])

print('how many unique indizes in galaxia: ',len(np.unique(indizes_g)))
print('independent volume elements in parsec isochrones: ',len(np.unique(indizes)))

print('Are all galaxia indizes covered by the isochrones?')

157747
indexing galaxia
how many unique indizes in galaxia:  26594
independent volume elements in parsec isochrones:  893259
Are all galaxia indizes covered by the isochrones?


In [10]:
problems = 0
total_fail=0
distances = []
unique = np.unique(indizes)
for i,item in enumerate(np.unique(indizes_g)):
    if i % 1000 == 0:
        print(i, len(np.unique(indizes_g)))
    try:
        assert(item in unique)
    except:
        problems += 1
        temp = y[np.where(indizes_g == item)]
        #We are staying with the same metallicity
        temp_cut = np.where(indizes%1000==item%1000)
        temp_x = x[0][temp_cut]
        temp_indizes = indizes[temp_cut]
        total_fail += len(temp)
        distance = (np.median(np.log10(temp['teff']))-temp_x['log_teff'])**2 + (np.median(temp['lum']) - temp_x['log_lum'])**2
        min_distance = min(distance)
        print(i,np.sqrt(min_distance))
        distances.append(np.sqrt(min_distance))
        indizes_g[np.where(indizes_g == item)] = temp_indizes[np.where(distance == min_distance)][0]

        
print("Problems with %d cells in galaxia with a total of %d stars" %(problems,total_fail))
        

0 26424
1000 26424
2000 26424
3000 26424
4000 26424
5000 26424
6000 26424
7000 26424
8000 26424
9000 26424
10000 26424
11000 26424
12000 26424
13000 26424
14000 26424
15000 26424
16000 26424
17000 26424
18000 26424
19000 26424
20000 26424
21000 26424
22000 26424
23000 26424
24000 26424
25000 26424
26000 26424
Problems with 0 cells in galaxia with a total of 0 stars
